### file load 

In [179]:
import pandas as pd
import numpy as np
import pymysql

# hid into load
path = 'VitalDB_SpHb_20211028.xlsx'
dfmain = pd.read_excel(path, sheet_name=0)
dfped = pd.read_excel(path, sheet_name=1)
df = dfmain.append(dfped, ignore_index=True)

# db connect
db = pymysql.connect(host='', port=, user='', passwd='', db='', charset='')

parid 목록 <br>
parid : 299 <br>

hb_abga : 472, 26339 <br>
hct_abga : 350, 26329

In [181]:
dfresult = pd.DataFrame(columns=list(dfmain.columns)+['Preop_T.bil','Time','Intraop_T.bil','POC_Hgb','POC_Hct'])
new_idx = 0
for idx, row in df.iterrows():
    hid = row['환자번호']
    orin = row['입실시간']
    orout = row['퇴실시간']
    
    # 6개월 이전의 preop lab 추출
    cur = db.cursor()
    sql = 'SELECT val FROM labs WHERE hid={} and parid = 299 and (dt BETWEEN "{}" - INTERVAL 6 MONTH AND "{}") ORDER BY dt DESC LIMIT 1'.format(hid,orin,orin)
    cur.execute(sql)
    if cur.rowcount == 1 :
        data = cur.fetchone()[0]
        df.loc[df.index==idx, 'Preop_T.bil'] = data
    
    # 수술 중 intraop lab 추출
    cur = db.cursor()
    sql = 'SELECT hid,parid,dt,val FROM labs WHERE hid={} and "{}" <= dt and dt <= "{}" and parid IN (299,350,472,26329,26339)'.format(hid, orin, orout)
    cur.execute(sql)
    vals = cur.fetchall()
    
    # intraop 없을 경우 기존행만 추가
    if len(vals) == 0 :
        dfresult = dfresult.append(df.iloc[idx],ignore_index=True)
    
    # intraop만큼 돌면서 해당 intraop의 시간 및 결과값 추가
    for data in range(len(vals)) :
        dfresult = dfresult.append(df.iloc[idx],ignore_index=True)

        labid = vals[data][1]
        time = vals[data][2]
        val = vals[data][3]
        # intraop T.bil
        if labid == 299 :
            dfresult.loc[dfresult.index==new_idx,'Time']=time
            dfresult.loc[dfresult.index==new_idx,'Intraop_T.bil']=val
        # intraop hb
        elif labid in [472,26339] :
            dfresult.loc[dfresult.index==new_idx,'Time']=time
            dfresult.loc[dfresult.index==new_idx,'POC_Hgb']=val
        # intraop hct
        elif labid in [350,26329] :
            dfresult.loc[dfresult.index==new_idx,'Time']=time
            dfresult.loc[dfresult.index==new_idx,'POC_Hct']=val
            
        new_idx += 1

### 시간이 동일할 경우 merge

In [156]:
col_list=['Intraop_T.bil','POC_Hgb','POC_Hct']

dfresult.fillna('', inplace=True)
dfresult = dfresult.astype('str')
new_dfresult = dfresult.groupby([i for i in list(set(dfresult.columns)) if i not in col_list], as_index=False)[col_list].sum()
new_dfresult = new_dfresult[list(dfresult.columns)]